In [164]:
#import warnings
#warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import viz

import scipy
from scipy import stats
import sklearn.impute as simpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler

from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression

from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector


import prepare

# Load the tips dataset.

Create a column named price_per_person. This should be the total bill divided by the party size.

In [165]:
df = sns.load_dataset("tips")
df["price_per_person"] = df["total_bill"]/df["size"]



dummy_df = pd.get_dummies(data=df[df.select_dtypes(include=['category']).columns.tolist()], drop_first=False)
dummy_df.drop(columns=["sex_Male","smoker_No","time_Lunch"])

df = df.drop(columns=(df.select_dtypes(include=['category']).columns.tolist()))
df = pd.concat([df,dummy_df],axis=1)

pred = "tip"
df

,total_bill,tip,size,price_per_person,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,1.01,2,8.495000,0,1,0,1,0,0,0,1,0,1
1,10.34,1.66,3,3.446667,1,0,0,1,0,0,0,1,0,1
2,21.01,3.50,3,7.003333,1,0,0,1,0,0,0,1,0,1
3,23.68,3.31,2,11.840000,1,0,0,1,0,0,0,1,0,1
4,24.59,3.61,4,6.147500,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,3,9.676667,1,0,0,1,0,0,1,0,0,1
240,27.18,2.00,2,13.590000,0,1,1,0,0,0,1,0,0,1
241,22.67,2.00,2,11.335000,1,0,1,0,0,0,1,0,0,1
242,17.82,1.75,2,8.910000,1,0,0,1,0,0,1,0,0,1


# Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
total price

In [166]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(df,test_size= .2, random_state=123)
train,validate = train_test_split(train,test_size= .25, random_state=123)

print(f"prepared df shape: {df.shape}")
print(f"train shape: {train.shape}")
print(f"validate shape: {validate.shape}")
print(f"test shape: {test.shape}")

not_pred=df.columns.drop(pred).tolist()


X_train,y_train = train[not_pred],train[pred]
X_validate,y_validate = validate[not_pred],validate[pred]
X_test,y_test = test[not_pred],test[pred]

prepared df shape: (244, 14)
train shape: (146, 14)
validate shape: (49, 14)
test shape: (49, 14)


# Use select k best to select the top 2 features for predicting tip amount. What are they?


In [167]:
kbest = SelectKBest(f_regression,k=2) ## k returns the best amount of features as entered
_ = kbest.fit(X_train,y_train)

kbest.pvalues_
kbest.scores_

kbest_resutls = pd.DataFrame(dict(p=kbest.pvalues_,f=kbest.scores_))
kbest_resutls
X_train.columns[kbest.get_support()]

X_train_transformed = pd.DataFrame(kbest.transform(X_train),columns=X_train.columns[kbest.get_support()],index=X_train.index)
X_train_transformed.head(3)

,total_bill,size
224,13.42,2.0
181,23.33,2.0
102,44.30,3.0


# Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [168]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
rfe = RFE(model,n_features_to_select=2)
rfe.fit(X_train,y_train)

pd.DataFrame(
    {"rfe_ranking":rfe.ranking_}
,index=X_train.columns)
X_train_transformed = pd.DataFrame(rfe.transform(X_train),columns=X_train.columns[rfe.get_support()],index=X_train.index)
X_train_transformed.head(3)

,day_Fri,day_Sun
224,1.0,0.0
181,0.0,1.0
102,0.0,0.0


In [171]:
var_ranks = rfe.ranking_
var_names = X_train.columns.tolist()

pd.DataFrame({'Var': var_names, 'Rank': var_ranks}).sort_values("Rank")

,Var,Rank
8,day_Fri,1
10,day_Sun,1
4,sex_Female,2
0,total_bill,3
2,price_per_person,4
9,day_Sat,5
12,time_Dinner,6
3,sex_Male,7
7,day_Thur,8
11,time_Lunch,9


# Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?
> - recursive works with minimal features to see which have the highest relation to prediction, while KBest keeps all features and sees which ones are highly correlated with each other, removing those. Long story short Kbest is more of a correlation answer and RFE is more which variable in a formula is most important.

In [ ]:
model = LinearRegression()
sfs = SequentialFeatureSelector(model, n_features_to_select=2)
sfs.fit(X_train, y_train)
X_train_transformed = pd.DataFrame(sfs.transform(X_train),columns=X_train.columns[sfs.get_support()],index=X_train.index)
X_train_transformed.head(3)

,total_bill,price_per_person
224,13.42,6.710000
181,23.33,11.665000
102,44.30,14.766667


# Write a function named select_kbest 
that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [191]:
def select_kbest(predictors_x,target_y,n_features):
    ''' 
    takes in the predictors (X) (X_train), the target (y) (y_train), and the number of features to select (k) 
    returns the names of the top k selected features based on the SelectKBest class.
    '''

    from sklearn.feature_selection import SelectKBest, f_regression

    kbest = SelectKBest(f_regression,k=n_features) ## k returns the best amount of features as entered
    _ = kbest.fit(predictors_x,target_y)

    kbest.pvalues_
    kbest.scores_

    kbest_resutls = pd.DataFrame(dict(p=kbest.pvalues_,f=kbest.scores_))
    kbest_resutls
    predictors_x.columns[kbest.get_support()]

    X_train_transformed = pd.DataFrame(kbest.transform(predictors_x),columns=predictors_x.columns[kbest.get_support()],index=predictors_x.index)
    X_train_transformed.head(3)
    print(X_train_transformed.columns.tolist())
    return(X_train_transformed)

In [192]:
select_kbest(X_train,y_train,2)


['total_bill', 'size']


,total_bill,size
224,13.42,2.0
181,23.33,2.0
102,44.30,3.0
164,17.51,2.0
73,25.28,2.0
...,...,...
233,10.77,2.0
6,8.77,2.0
7,26.88,4.0
115,17.31,2.0


# Write a function named rfe 
that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [199]:
def rfe(predictors_x,target_y,n_features):
    ''' 
    takes in the predictors (X) (predictors_x), the target (y) (target_y), and the number of features to select (k) 
    returns the names of the top k selected features based on the Recursive Feature Elimination class. and a ranked df
    '''

    from sklearn.linear_model import LinearRegression
    from sklearn.feature_selection import RFE

    model = LinearRegression()
    rfe = RFE(model,n_features_to_select=n_features)
    rfe.fit(predictors_x,target_y)

    print(pd.DataFrame({"rfe_ranking":rfe.ranking_},index=predictors_x.columns).sort_values("rfe_ranking")[:n_features])
    X_train_transformed = pd.DataFrame(rfe.transform(predictors_x),columns=predictors_x.columns[rfe.get_support()],index=predictors_x.index)
    X_train_transformed.head(3)

    var_ranks = rfe.ranking_
    var_names = predictors_x.columns.tolist()

    rfe_ranked = pd.DataFrame({'Var': var_names, 'Rank': var_ranks}).sort_values("Rank")
    
    return rfe_ranked

In [200]:
rfe(X_train,y_train,2)

         rfe_ranking
day_Fri            1
day_Sun            1


,Var,Rank
8,day_Fri,1
10,day_Sun,1
4,sex_Female,2
0,total_bill,3
2,price_per_person,4
9,day_Sat,5
12,time_Dinner,6
3,sex_Male,7
7,day_Thur,8
11,time_Lunch,9


# Load the swiss dataset and use all the other features to predict Fertility. 
Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [214]:
from pydataset import data
from sklearn.model_selection import train_test_split

df = data("swiss")

train,test = train_test_split(df,test_size= .2, random_state=123)
train,validate = train_test_split(train,test_size= .25, random_state=123)

print(f"prepared df shape: {df.shape}")
print(f"train shape: {train.shape}")
print(f"validate shape: {validate.shape}")
print(f"test shape: {test.shape}\n\n")

pred = "Fertility"
not_pred=df.columns.drop(pred).tolist()


X_train,y_train = train[not_pred],train[pred]
X_validate,y_validate = validate[not_pred],validate[pred]
X_test,y_test = test[not_pred],test[pred]
print("-----------------------------\n")
select_kbest(X_train,y_train,3)
print("^^^^^^^^^^^^KBest^^^^^^^^^^\n")
print("-----------------------------\n")
print("vvvvvvvvvvvvvRFEvvvvvvvvvvvvv")
rfe(X_train,y_train,3)


prepared df shape: (47, 6)
train shape: (27, 6)
validate shape: (10, 6)
test shape: (10, 6)


-----------------------------

['Examination', 'Catholic', 'Infant.Mortality']
^^^^^^^^^^^^KBest^^^^^^^^^^

-----------------------------

vvvvvvvvvvvvvRFEvvvvvvvvvvvvv
                  rfe_ranking
Examination                 1
Education                   1
Infant.Mortality            1


,Var,Rank
1,Examination,1
2,Education,1
4,Infant.Mortality,1
0,Agriculture,2
3,Catholic,3
